In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from pydantic import BaseModel, Field
from langgraph.graph.message import add_messages

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

# ----- STATE -----
class ChatGraphState(TypedDict):
    old_conversation: list[BaseMessage]
    latest_chat: str
    tool_calls: list[dict]  # to store tool call details

# ----- TOOL SCHEMA -----
class SearchDatabaseInput(BaseModel):
    old_conversation: list[BaseMessage] = Field(..., description="The old conversation")
    latest_chat: str = Field(..., description="The latest user message")

class CreateTicketInput(BaseModel):
    customer_name:str=Field(..., description=" user name who has issue and need assistance")
    contact:str=Field(..., description=" user contact number who has issue and need assistance")
    description:str=Field(..., description=" user issue detailed description ")

def search_database(old_conversation: list[BaseMessage], latest_chat: str):
    # Convert messages to text
    conversation_text = "\n".join([msg.content for msg in old_conversation])
    return {"results": f"Based on previous conversation:\n{conversation_text}\nAnswer to: {latest_chat}"}

def create_ticket(CreateTicketInput):
    # Convert messages to text
    print("desc",{description})
    return {"results": f"we have create ticket for you {customer_name},we will contact you seeon at {contact}"}

# ----- SAMPLE CONVERSATION -----
messages = [
    {"role": "user", "content": "Hi, my order hasn’t arrived yet. Can you help?"},
    {"role": "agent", "content": "Of course! Could you please provide your order number so I can check the status?"},
    {"role": "user", "content": "Yes, it’s #ORD7821."},
    {"role": "agent", "content": "Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM."},
    {"role": "user", "content": "Great, thanks for the update!"},
    {"role": "agent", "content": "You’re welcome! Is there anything else I can help you with today?"},
    {"role": "user", "content": "No, that’s all. Have a nice day!"},
    {"role": "agent", "content": "You too! 😊"}
]

# Convert dict messages to BaseMessage objects
converted_messages = []
for msg in messages:
    if msg["role"] == "user":
        converted_messages.append(HumanMessage(content=msg["content"]))
    else:
        converted_messages.append(AIMessage(content=msg["content"]))

# ----- NODE -----
def gen_reply(state: ChatGraphState):
    print("STATE:", state)

    llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
    llm_with_tools = llm.bind_tools([search_database,create_ticket], tool_choice="auto")

    # Invoke LLM
    response = llm_with_tools.invoke([HumanMessage(content=state["latest_chat"])])

    tool_call_details = []

    # If tool was called
    if hasattr(response, "tool_calls") and response.tool_calls:
        for call in response.tool_calls:
            if call["name"] == "search_database":
                args = {"old_conversation": state["old_conversation"], "latest_chat": state["latest_chat"]}
                result = search_database(**args)
                tool_call_details.append({"tool_name": call["name"], "args": args, "result": result})

                # Wrap tool result as AIMessage
                response = AIMessage(content=result["results"])

    # Add the LLM response to the conversation
    return {
        "old_conversation": add_messages(state["old_conversation"], [response]),
        "latest_chat": state["latest_chat"],
        "tool_calls": tool_call_details
    }

# ----- GRAPH -----
graph = StateGraph(ChatGraphState)
graph.add_node("gen_reply", gen_reply)
graph.add_edge(START, "gen_reply")
graph.add_edge("gen_reply", END)

chatbot = graph.compile()

# ----- INITIAL STATE -----
initial_state = {
    "old_conversation": converted_messages,
    "latest_chat": "Hello, I forgot my password?",
    "tool_calls": []
}

# Run chatbot
result = chatbot.invoke(initial_state)
print("FINAL RESULT:", result)


STATE: {'old_conversation': [HumanMessage(content='Hi, my order hasn’t arrived yet. Can you help?', additional_kwargs={}, response_metadata={}), AIMessage(content='Of course! Could you please provide your order number so I can check the status?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yes, it’s #ORD7821.', additional_kwargs={}, response_metadata={}), AIMessage(content='Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Great, thanks for the update!', additional_kwargs={}, response_metadata={}), AIMessage(content='You’re welcome! Is there anything else I can help you with today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='No, that’s all. Have a nice day!', additional_kwargs={}, response_metadata={}), AIMessage(content='You too! 😊', additional_kwargs={}, response_metadata={})], 'latest_chat': 'Hello, I forgot my password?'

In [7]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from pydantic import BaseModel, Field
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from copy import deepcopy

# ----- STATE -----
class ChatGraphState(TypedDict):
    old_conversation: list[BaseMessage]
    latest_chat: str
    tool_calls: list[dict]  # store tool call info
    decision: str  # 'search' or 'ticket'
    contact_info: str  # for ticket creation
    final_reply: str  # final AI reply

# ----- TOOLS -----
def search_database(old_conversation: list[BaseMessage], latest_chat: str):
    conversation_text = "\n".join([msg.content for msg in old_conversation])
    result = {"results": f"Search result based on conversation:\n{conversation_text}\nAnswer to: {latest_chat}"}
    return result

def create_ticket(customer_name:str, contact:str, description:str):
    result = {"results": f"Ticket created for {customer_name} with contact {contact}. Issue: {description}"}
    return result

# ----- SAMPLE CONVERSATION -----
messages = [
    {"role": "user", "content": "Hi, I forgot my password."},
    {"role": "agent", "content": "Hello! Please provide your contact number."}
]

# Convert dict messages to BaseMessage objects
converted_messages = []
for msg in messages:
    if msg["role"] == "user":
        converted_messages.append(HumanMessage(content=msg["content"]))
    else:
        converted_messages.append(AIMessage(content=msg["content"]))

# ----- LLM -----
llm_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

# ----- AGENTS -----
def search_agent(state: ChatGraphState):
    result = search_database(state["old_conversation"], state["latest_chat"])
    response = AIMessage(content=result["results"])
    state["old_conversation"] = add_messages(state["old_conversation"], [response])
    state["tool_calls"].append({
        "tool": "search_database",
        "args": {"latest_chat": state["latest_chat"]},
        "result": result
    })
    return state

def ticket_agent(state: ChatGraphState):
    if not state.get("contact_info"):
        user_msg = HumanMessage(content="Please provide your contact number for ticket creation.")
        state["old_conversation"] = add_messages(state["old_conversation"], [user_msg])
        state["contact_info"] = "123-456-7890"  # dummy input
    result = create_ticket("John Doe", state["contact_info"], state["latest_chat"])
    response = AIMessage(content=result["results"])
    state["old_conversation"] = add_messages(state["old_conversation"], [response])
    state["tool_calls"].append({
        "tool": "create_ticket",
        "args": {"latest_chat": state["latest_chat"], "contact": state["contact_info"]},
        "result": result
    })
    return state

def reply_generator(state: ChatGraphState):
    # Generate final user-friendly response from conversation
    prompt = f"""
Format a user-friendly reply based on the conversation and tool results:
{chr(10).join([msg.content for msg in state['old_conversation']])}
"""
    ai_response = llm_model.invoke([HumanMessage(content=prompt)])
    state["final_reply"] = ai_response.content
    response_msg = AIMessage(content=ai_response.content)
    state["old_conversation"] = add_messages(state["old_conversation"], [response_msg])
    return state

def decision_agent(state: ChatGraphState):
    # LLM decides 'search' or 'ticket'
    prompt = f"""
You are an assistant that decides whether to search the database or create a ticket.
Conversation so far:
{chr(10).join([msg.content for msg in state['old_conversation']])}

Latest user message:
{state['latest_chat']}

Respond with ONLY one word: 'search' or 'ticket'.
"""
    llm_response = llm_model.invoke([HumanMessage(content=prompt)])
    decision_text = llm_response.content.strip().lower()
    if decision_text not in ["search", "ticket"]:
        decision_text = "search"
    state["decision"] = decision_text
    print("DECISION:", decision_text)

    # Optionally generate an initial reply
    initial_reply = f"Processing your request using {decision_text}..."
    state["old_conversation"] = add_messages(state["old_conversation"], [AIMessage(content=initial_reply)])

    # Call the chosen agent
    if decision_text == "search":
        state = search_agent(deepcopy(state))
    else:
        state = ticket_agent(deepcopy(state))

    # Then call reply generator
    state = reply_generator(state)
    return state

# ----- GRAPH -----
graph = StateGraph(ChatGraphState)
graph.add_node("decision_agent", decision_agent)
graph.add_node("search_agent", search_agent)
graph.add_node("ticket_agent", ticket_agent)
graph.add_node("reply_generator", reply_generator)

graph.add_edge(START, "decision_agent")
graph.add_edge("decision_agent", END)

# ----- INITIAL STATE -----
initial_state = {
    "old_conversation": converted_messages,
    "latest_chat": "98011454",
    "tool_calls": [],
    "decision": "",
    "contact_info": "",
    "final_reply": ""
}

# Run chatbot
result = graph.compile().invoke(initial_state)

# ----- FINAL OUTPUT -----
print("FINAL STATE CONVERSATION:")
for msg in result["old_conversation"]:
    print(msg.content)
print("\nTOOL CALL DETAILS:")
for call in result["tool_calls"]:
    print(call)
print("\nFINAL REPLY:", result["final_reply"])


DECISION: search
FINAL STATE CONVERSATION:
Hi, I forgot my password.
Hello! Please provide your contact number.
Processing your request using search...
Search result based on conversation:
Hi, I forgot my password.
Hello! Please provide your contact number.
Processing your request using search...
Answer to: 98011454
Hello there!

Thank you for providing your contact number. We have processed your request and found the information you need to reset your password. To ensure your security, we will send the password reset instructions to the phone number ending in 454. Please follow the steps provided in the message to regain access to your account.

If you don't receive the instructions within the next few minutes, or if you need further assistance, don't hesitate to reach out to our support team.

Best regards,
[Your Support Team]

TOOL CALL DETAILS:
{'tool': 'search_database', 'args': {'latest_chat': '98011454'}, 'result': {'results': 'Search result based on conversation:\nHi, I forgot 